In [ ]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2

# load general packages
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import time
plt.style.use('seaborn-whitegrid')

# Exercise 2: EGM

Consider the model
$$\begin{align*}
V_{t}(M_{t}) & = & \max_{C_{t}}\left\{ \frac{C_{t}^{1-\rho}}{1-\rho}+\beta\mathbb{E}_{t}\left[V_{t+1}(M_{t+1})\right]\right\} \\
 & \text{s.t.}\\
M_{t+1} & = & R(M_{t}-C_{t})+Y_{t+1}\\
Y_{t+1} & = & \exp(\xi_{t+1})\\
\xi_{t+1} & = & \mathcal{N}(\mu,\sigma_{\xi}^{2})\\
A_{t}=M_{t}-C_{t} & \geq & 0
\end{align*}$$

for $T=10$, $\beta=0.98$, $\rho=0.5$, $R=1.0/\beta$, $M_1=10$ and $\sigma_{\xi}=0.2$.

### 1. Solve the model using time iteration. Remember that optimal consumption must satisfy the Euler-equation:
$$\begin{align*}
u'(C_t) = \beta R \mathbb{E}_t[u'(C_{t+1}^{\star}(M_{t+1}))]
\end{align*}$$

In [ ]:
# Import EGM model
import Exercise_2 as ex2

# Setup
par_EGM = ex2.setup()

# Solve
sol_EGM = ex2.solve_EGM(par_EGM, vector=False)

In [ ]:
# Import time iterations model
import Exercise_1 as ex1

# Setup
par_TI = ex1.setup()

# Solve
sol_TI = ex1.solve_ti(par_TI)

In [ ]:
# Print consumption function
fig = plt.figure(figsize=(8,5))# figsize is in inches...
ax = fig.add_subplot(1,1,1)
for t in range(par_EGM.T):
    ax.plot(sol_EGM.M[:,t],sol_EGM.C[:,t],  label=f"t = {t + 1}, EGM",  linestyle='-')
    ax.plot(par_TI.grid_M,sol_TI.C[:,t],    label=f"t = {t + 1}, TI",   linestyle='--')
ax.set_xlabel(f"$M_t$")
ax.set_ylabel(f"$C_t$")
ax.set_title(f'Consumption function')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

### 2. Simulate a panel of *N=10000* household for *T=10* periods and plot the mean of $C_t$. Everybody should be initialized with $W_t = 1.5$.

In [ ]:
sim = ex2.simulate(par_EGM,sol_EGM)

def figure_sim(simC,par):
    fig = plt.figure(figsize=(10,4))# figsize is in inches...
    t_grid = [t for t in range(1,par.T+1)]    
    ax = fig.add_subplot(1,1,1)
    ax.plot(t_grid,np.mean(sim.C,0),'-o')
    ax.set_xlabel(f"t")
    ax.set_ylabel(f"Mean of consumption")
    ax.set_ylim(bottom=0.5,top=1.5)
    plt.show()
    
figure_sim(sim.C,par_EGM)

### 3. How does the speed of the EGM procedure compare with the speed of time iterations
(Optional) Could you write a vectorized version of EGM to speed it up? (i.e. without no loop over $a_t$)

In [ ]:
# Time iterations
par_TI = ex1.setup()
print("Time spent on vectorized time iterations:")
%timeit -n 10 sol_TI = ex1.solve_ti(par_TI)

# EGM with loop
par_EGM = ex2.setup()
print("Time spent on EGM with loop:")
%timeit -n 10 sol_EGM = ex2.solve_EGM(par_EGM, vector=False)

# # Vectorized EGM
# par_EGM = ex2.setup()
# print("Time spent on vectorized EGM:")
# %timeit -n 10 sol_EGM_vec = ex2.solve_EGM(par_EGM, vector=True)